In [1]:
import tensorflow as tf
import os
import numpy as np
import pickle

CIFAR_DIR = "cifar-10-batches-py"

In [2]:
def load_data(fileName):
    """从文件中读取数据"""
    with open(fileName, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']
    
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        # 将数据缩放到[-1, 1]
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        
        self._indicator = 0  # 当前遍历到的位置
        if self._need_shuffle:
            self._shuffle_data()
        
        
    """打乱训练数据集"""
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    """返回batch_size个样本"""
    def next_batch (self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch_size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    
"""加载数据"""  
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)





(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
"""残差连接块"""
def residual_block(x, output_channel):
    """残差第一个分支"""
    # 如果output_channel = 2 * input_channel 要进行降采样
    input_channel = x.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True  #是否增加输出channel数
        strides = (2, 2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x, output_channel, (3, 3), activation=tf.nn.relu, strides=strides, padding='same', name='conv1')
    conv2 = tf.layers.conv2d(conv1, output_channel, (3, 3), strides=(1, 1), padding='same', activation=tf.nn.relu, name='conv2')
    """残差第二个分支"""
    if increase_dim:
        # [None, image_width, image_height, channel] -> [,,,2 * channel]
        pooled_x = tf.layers.max_pooling2d(x, (2, 2), (2, 2), padding='valid')
        
        padded_x = tf.pad(pooled_x, [[0,0], [0,0], [0,0], [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    
    output_x = conv2 + padded_x
    return output_x

"""ResNet 实现"""
def res_net(x, # 输入
            num_residual_blocks, # 每一层需要多少个残差连接块 eg:[3, 4, 6, 3]
            num_filter_base, # 最初的通道数目
            class_num):
    num_subsampling = len(num_residual_blocks) # 有几个残差块
    layers = []
    # x: [None, width, height, channel] -> [width, height, channel]
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x, num_filter_base, (3, 3), padding='same', strides=(1, 1), activation=tf.nn.relu, name='conv0')
        layers.append(conv0)
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d"% (sample_id, i)):
                conv = residual_block(layers[-1],num_filter_base * (2 ** sample_id))
                layers.append(conv)
                
    #测试实际的神经元图大小是否等于预测的神经元图的大小
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] == [input_size[0] / multiplier, input_size[1] / multiplier, num_filter_base * multiplier]
    
    with tf.variable_scope('fc'):
        # layer[-1].shape : [None, width, height, channel]
        # kernal_size: image_width, image_height
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]
    
    
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

In [4]:
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32 * 32
x_image = tf.transpose(x_image,  perm=[0, 2, 3, 1])



y_ = res_net(x_image, (2, 3, 2), 32, 10)

"""交叉熵损失函数"""
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_


#indices
predict = tf.arg_max(y_, 1)
corrent_preicttion = tf.equal(predict, y)
#准确率
accuracy = tf.reduce_mean(tf.cast(corrent_preicttion, tf.float64))


#梯度下降
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use `argmax` instead


In [5]:
# 初始化函数
init = tf.global_variables_initializer() 
batch_size = 20
batch_steps = 10000
test_steps = 100
#实例化session
with tf.Session() as sess:
    sess.run(init)
    for i in range(batch_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
#         每5000次进行一次测试
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' %(i+1, test_acc))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv0/conv0/Conv2D (defined at <ipython-input-3-48187009dbc1>:38)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape, conv0/conv0/kernel/read)]]
	 [[{{node sparse_softmax_cross_entropy_loss/value/_15}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1024_sparse_softmax_cross_entropy_loss/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'conv0/conv0/Conv2D', defined at:
  File "E:\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod